<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/QM-DM-ClassifierEig-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np
import tensorflow as tf
import qmc.tf.layers as layers
import qmc.tf.models as models


# Digits

In [3]:
X,y = load_digits(return_X_y=True)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, stratify=y, random_state=42)

print(X_train.shape)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_bin = tf.reshape (tf.keras.backend.one_hot(y_train, 10), (-1,10))
y_test_bin = tf.reshape (tf.keras.backend.one_hot(y_test, 10), (-1,10))
print(y_train.shape)
print(y_train_bin.shape)

(1203, 64)
(1203, 1)
(1203, 10)


In [4]:
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
fm_y = layers.QFeatureMapOneHot(num_classes=10)
qmc_dig = models.QMClassifier(fm_x=fm_x, fm_y=fm_y, dim_x=150, dim_y=10)
qmc_dig.compile()
qmc_dig.fit(X_train, y_train, epochs=1)
out = qmc_dig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

38/38 [==============================] - 3s 79ms/step
0.9377104377104377


In [5]:
qmc_sgd_dig = models.QMClassifierSGD(input_dim=64, dim_x=150, dim_y=10,
                                     num_eig=10, gamma=0.375, random_state=17)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
qmc_sgd_dig.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmc_sgd_dig.fit(X_train, y_train_bin, epochs=10)
out = qmc_sgd_dig.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 0s 1ms/step - loss: 1.3829
Epoch 2/10
38/38 [==============================] - 0s 1ms/step - loss: 0.3436
Epoch 3/10
38/38 [==============================] - 0s 1ms/step - loss: 0.2404
Epoch 4/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1909
Epoch 5/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1529
Epoch 6/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1316
Epoch 7/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1170
Epoch 8/10
38/38 [==============================] - 0s 1ms/step - loss: 0.1038
Epoch 9/10
38/38 [==============================] - 0s 1ms/step - loss: 0.0883
Epoch 10/10
38/38 [==============================] - 0s 1ms/step - loss: 0.0844
0.9713804713804713


In [6]:
inputs = tf.keras.Input(shape=(64,))
fm_x = layers.QFeatureMapRFF(64, dim=150, gamma=0.375, random_state=17)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc = layers.QMeasureDMClassifEig(dim_x=150, dim_y=10, eig_out=10, num_eig=10)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmc = tf.keras.Model(inputs=inputs, outputs=probs)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
qmdmc.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy())
qmdmc.fit(X_train, y_train_bin, epochs=10)
out = qmdmc.predict(X_test)
print(accuracy_score(y_test, np.argmax(out, axis=1)))

Epoch 1/10
38/38 [==============================] - 1s 2ms/step - loss: 1.2683
Epoch 2/10
38/38 [==============================] - 0s 2ms/step - loss: 0.3484
Epoch 3/10
38/38 [==============================] - 0s 2ms/step - loss: 0.2048
Epoch 4/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1637
Epoch 5/10
38/38 [==============================] - 0s 3ms/step - loss: 0.1321
Epoch 6/10
38/38 [==============================] - 0s 2ms/step - loss: 0.1110
Epoch 7/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0973
Epoch 8/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0896
Epoch 9/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0791
Epoch 10/10
38/38 [==============================] - 0s 2ms/step - loss: 0.0706
0.9764309764309764


# MNIST

In [8]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)
transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


In [9]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = tf.keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(input_dim, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.DMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size)

    qmkdc1 = models.DMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, 
                                      random_state=random_state)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [58]:
input_dim = 784
component_dim = 128
num_classes = 10
gamma = 2**-5
random_state=0
num_eig=60
batch_size=32
epochs = 10
model_mnist = models.DMKDClassifierSGD(input_dim=input_dim, dim_x=component_dim,                                                          num_eig=num_eig, 
                                       num_classes=num_classes, 
                                       gamma=gamma,random_state=random_state)


In [59]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model_mnist.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

In [60]:
history = model_mnist.fit(X_train, y_train_bin, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.9126 - accuracy: 0.7807
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3224 - accuracy: 0.9449
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2528 - accuracy: 0.9615
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2181 - accuracy: 0.9672
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2022 - accuracy: 0.9718
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1785 - accuracy: 0.9755
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1646 - accuracy: 0.9788
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1520 - accuracy: 0.9813
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1441 - accuracy: 0.9831
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.139

In [61]:
out = model_mnist.predict(X_test, batch_size=batch_size)
print(accuracy_score(y_test, out.argmax(axis=1)))

0.9695


In [62]:
inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x(inputs)
ones = tf.fill((tf.shape(inputs)[0], 1), 1.0)
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig * 2, num_eig=num_eig)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmc = tf.keras.Model(inputs=inputs, outputs=probs)

In [63]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
qmdmc.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])
history = qmdmc.fit(X_train, y_train_bin, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 2.1461 - accuracy: 0.2668
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 1.5701 - accuracy: 0.5781
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 1.2851 - accuracy: 0.6878
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 1.0990 - accuracy: 0.7506
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.9772 - accuracy: 0.7877
Epoch 6/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.8807 - accuracy: 0.8099
Epoch 7/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.8059 - accuracy: 0.8243
Epoch 8/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.7429 - accuracy: 0.8347
Epoch 9/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6981 - accuracy: 0.8395
Epoch 10/10
1875/1875 [==============================] - 12s 6ms/step - l

In [64]:
out = qmdmc.predict(X_test, batch_size=batch_size)
print(accuracy_score(y_test, out.argmax(axis=1)))

0.854
